In [1]:
# Generate `Objeto de Gasto` catalog

## File ingestion

In [2]:
from pandas import ExcelFile, read_excel, DataFrame
from slugify import slugify

In [3]:
cd /home/loic/repos/mexico

/home/loic/repos/mexico


In [7]:
catalog_file = ExcelFile('objeto_del_gasto.catalog.xlsx')

In [8]:
catalog = {}

for sheet in catalog_file.sheet_names:
    name = slugify(sheet, separator='_')
    catalog[name] = catalog_file.parse(sheet).dropna()
    message = 'Loaded sheet {sheet} into "{name}" ({nb} lines)'
    parameters = dict(sheet=sheet, name=name, nb=len(catalog[name]))
    print(message.format(**parameters))
    print('Columns =', list(catalog[name].columns))

Loaded sheet Concatenated into "concatenated" (460 lines)
Columns = ['CAPITULO', 'CONCEPTO', 'PARTIDA_GENERICA', 'PARTIDA_ESPECIFICA', 'DESCRIPCION']
Loaded sheet CAPITULO into "capitulo" (9 lines)
Columns = ['CAPITULO', 'DESCRIPCION']
Loaded sheet CONCEPTO into "concepto" (88 lines)
Columns = ['CONCEPTO', 'DESCRIPCION']
Loaded sheet PARTIDA GENERICA into "partida_generica" (355 lines)
Columns = ['PARTIDA_GENERICA', 'DESCRIPCION']
Loaded sheet PARTIDA ESPECÍFICA into "partida_especifica" (668 lines)
Columns = ['PARTIDA_ESPECIFICA', 'DESCRIPCION']


## Quality assurance

The "Concatenated" and the "PARTIDA ESPECÍFICA" sheets must be the same.

In [61]:
catalog['partida_especifica']['PARTIDA_ESPECIFICA'] =  catalog['partida_especifica']['PARTIDA_ESPECIFICA'].astype(int)
especifica_1 = catalog['partida_especifica'].set_index('PARTIDA_ESPECIFICA').sort_index()
print(especifica_1.info())
especifica_1.head(n=5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460 entries, 11101 to 99101
Data columns (total 1 columns):
DESCRIPCION    460 non-null object
dtypes: object(1)
memory usage: 7.2+ KB
None


,DESCRIPCION
PARTIDA_ESPECIFICA,
11101,(Derogada)
11201,Haberes
11301,Sueldos base
11401,Retribuciones por adscripción en el extranjero
12101,Honorarios


In [65]:
catalog['concatenated']['PARTIDA_ESPECIFICA'] =  catalog['concatenated']['PARTIDA_ESPECIFICA'].astype(int)
especifica_2 = catalog['concatenated'][['PARTIDA_ESPECIFICA', 'DESCRIPCION']].set_index('PARTIDA_ESPECIFICA').sort_index()
print(especifica_2.info())
especifica_2.head(n=5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460 entries, 11101 to 99101
Data columns (total 1 columns):
DESCRIPCION    460 non-null object
dtypes: object(1)
memory usage: 7.2+ KB
None


,DESCRIPCION
PARTIDA_ESPECIFICA,
11101,(Derogada)
11201,Haberes
11301,Sueldos base
11401,Retribuciones por adscripción en el extranjero
12101,Honorarios


In [67]:
comparaison = especifica_1 == especifica_2

In [69]:
comparaison.all()

DESCRIPCION    True
dtype: bool

Okay, everything seems to be coherent, after I get rid of the hidden rows in the `Concatenated` sheet. 

## Lookup table

Now I need to generate lookup tables for `CAPITULO`, `CONCEPTO`, `PARTIDA_GENERICA` and `PARTIDA_ESPECIFICA`, or rather a lookup function which gives me the description from the ID. It's as simple as using the dataframe indexing, like so:

In [76]:
especifica_1.loc[11101]

DESCRIPCION    (Derogada)
Name: 11101, dtype: object

In [12]:
from os.path import join

def generate_catalog(file):
    
    new_columns = {}
    catalog_file = ExcelFile(file)
    INDEX_COLUMN = 0
    
    for sheet in catalog_file.sheet_names:
        if sheet != 'Concatenated':
            name = slugify(sheet, separator='_')
            output = join('objeto_del_gasto.catalog', name + '.csv')

            df = catalog_file.parse(sheet).dropna()
            index = df.columns[INDEX_COLUMN]

            df[index] =  df[index].astype(int)
            df.set_index(index, inplace=True)
            df.sort_index(inplace=True)
            
            new_columns[name] = df
            df.to_csv(output)
            
            message = 'Loaded sheet {sheet} into "{name}" ({nb} lines)'
            parameters = dict(sheet=sheet, name=name, nb=len(catalog[name]))

            print(message.format(**parameters))
            print('Columns =', list(catalog[name].columns))
            print('Saved to', output)
            
    return new_columns

In [13]:
catalog = generate_catalog('objeto_del_gasto.catalog.xlsx')

Loaded sheet CAPITULO into "capitulo" (9 lines)
Columns = ['CAPITULO', 'DESCRIPCION']
Saved to objeto_del_gasto.catalog/capitulo.csv
Loaded sheet CONCEPTO into "concepto" (88 lines)
Columns = ['CONCEPTO', 'DESCRIPCION']
Saved to objeto_del_gasto.catalog/concepto.csv
Loaded sheet PARTIDA GENERICA into "partida_generica" (355 lines)
Columns = ['PARTIDA_GENERICA', 'DESCRIPCION']
Saved to objeto_del_gasto.catalog/partida_generica.csv
Loaded sheet PARTIDA ESPECÍFICA into "partida_especifica" (668 lines)
Columns = ['PARTIDA_ESPECIFICA', 'DESCRIPCION']
Saved to objeto_del_gasto.catalog/partida_especifica.csv


In [94]:
catalog['capitulo'].loc[1000]

DESCRIPCION    Servicios personales
Name: 1000, dtype: object

In [115]:
catalog['partida_especifica'].loc[21101]

DESCRIPCION    Materiales y útiles de oficina
Name: 21101, dtype: object

## Read pre-processed catalog files

In [111]:
from pandas import read_csv
from os import listdir
from os.path import join

def load_catalogs(folder):
    
    catalogs = {}
    files = listdir(folder)
    
    for file in files:
        name = file.split('.')[0]
        print('Loading', name)
        filepath = join(folder, file)
        
        catalogs[name] = read_csv(filepath)
        index_column = catalogs[name].columns[0]
        catalogs[name].set_index(index_column, inplace=True)
    
        print(catalogs[name].info(), '\n')
    
    return catalogs

catalogs = load_catalogs('objeto_del_gasto.catalog')

Loading partida_generica
<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 111 to 991
Data columns (total 1 columns):
DESCRIPCION    351 non-null object
dtypes: object(1)
memory usage: 5.5+ KB
None 

Loading capitulo
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 1000 to 9000
Data columns (total 1 columns):
DESCRIPCION    9 non-null object
dtypes: object(1)
memory usage: 144.0+ bytes
None 

Loading concepto
<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 1100 to 9900
Data columns (total 1 columns):
DESCRIPCION    88 non-null object
dtypes: object(1)
memory usage: 1.4+ KB
None 

Loading partida_especifica
<class 'pandas.core.frame.DataFrame'>
Int64Index: 460 entries, 11101 to 99101
Data columns (total 1 columns):
DESCRIPCION    460 non-null object
dtypes: object(1)
memory usage: 7.2+ KB
None 



In [116]:
catalogs['capitulo'].loc[2000]

DESCRIPCION    Materiales y suministros
Name: 2000, dtype: object

## Test splitting the data

In [199]:
from pandas import DataFrame
from numpy import nan, int32

ids = catalogs['capitulo'].sample(n=6).reset_index()
del ids['DESCRIPCION']
ids.loc[5] = nan
ids['CAPITULO'] = ids['CAPITULO'].astype(str, inplace=True)
print(ids.info())
ids

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
CAPITULO    6 non-null object
dtypes: object(1)
memory usage: 128.0+ bytes
None


,CAPITULO
0,7000.0
1,5000.0
2,4000.0
3,9000.0
4,2000.0
5,nan


In [198]:
str(nan)

'nan'

In [167]:
df = DataFrame()
df

""


In [170]:
ids['CAPITULO']

0    9000
1    6000
2    8000
3    4000
4    3000
Name: CAPITULO, dtype: int64

In [178]:
description = catalog['capitulo'].loc[ids['CAPITULO']]['DESCRIPCION'].reset_index()
del description['CAPITULO']
description

,DESCRIPCION
0,Deuda publica
1,Inversion publica
2,Participaciones y aportaciones
3,"Transferencias, asignaciones, subsidios y otra..."
4,Servicios generales


In [179]:
from pandas import concat

concat([description, ids], axis=1)

,DESCRIPCION,CAPITULO
0,Deuda publica,9000
1,Inversion publica,6000
2,Participaciones y aportaciones,8000
3,"Transferencias, asignaciones, subsidios y otra...",4000
4,Servicios generales,3000


In [182]:
description = catalog['capitulo'].loc[ids['CAPITULO']].reset_index()
description

,CAPITULO,DESCRIPCION
0,9000,Deuda publica
1,6000,Inversion publica
2,8000,Participaciones y aportaciones
3,4000,"Transferencias, asignaciones, subsidios y otra..."
4,3000,Servicios generales


In [5]:
from pandas import read_csv

merged = read_csv('pipeline.out/iteration-before-holiday/mexican_federal_budget.merged.csv')
merged.head(n=5)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (4,11,16,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Actividad Institucional,Adefas,Aprobado,Ciclo,Clave de cartera,Descripción de Fuente de Financiamiento,Descripción de Función,Descripción de Grupo Funcional,Descripción de Objeto del Gasto,Descripción de Programa Presupuestario,...,Modalidad del Programa presupuestario,Modificado,Objeto del Gasto,Pagado,Programa Presupuestario,Ramo,Reasignacion,Subfunción,Tipo de Gasto,Unidad Responsable
0,4,NaN,9.930500e+07,2016,0,Recursos fiscales,Legislación,Gobierno,Obra pública en bienes propios,Mantenimiento de Infraestructura,...,K,NaN,6200.0,NaN,27.0,1,0.0,1,3.0,100
1,4,NaN,1.003938e+09,2016,0,Recursos fiscales,Legislación,Gobierno,Remuneraciones al personal de carácter permanente,Actividades derivadas del trabajo legislativo,...,R,NaN,1100.0,NaN,1.0,1,0.0,1,1.0,100
2,4,NaN,9.651971e+08,2016,0,Recursos fiscales,Legislación,Gobierno,Remuneraciones al personal de carácter transit...,Actividades derivadas del trabajo legislativo,...,R,NaN,1200.0,NaN,1.0,1,0.0,1,1.0,100
3,4,NaN,1.024815e+09,2016,0,Recursos fiscales,Legislación,Gobierno,Remuneraciones adicionales y especiales,Actividades derivadas del trabajo legislativo,...,R,NaN,1300.0,NaN,1.0,1,0.0,1,1.0,100
4,4,NaN,3.814606e+08,2016,0,Recursos fiscales,Legislación,Gobierno,Seguridad social,Actividades derivadas del trabajo legislativo,...,R,NaN,1400.0,NaN,1.0,1,0.0,1,1.0,100


In [6]:
objeto_raw = merged[['Objeto del Gasto',]]
objeto_raw.head(n=5)

,Objeto del Gasto
0,6200.0
1,1100.0
2,1200.0
3,1300.0
4,1400.0


In [7]:
len(objeto_raw)

1325512

In [8]:
objeto = objeto_raw.dropna()
len(objeto.dropna())

1325511

In [9]:
objeto.head(n=5)

,Objeto del Gasto
0,6200.0
1,1100.0
2,1200.0
3,1300.0
4,1400.0


In [10]:
objeto['Objeto del Gasto'] = objeto['Objeto del Gasto'].astype(int)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Split the data line by line

Missing values are a pain in the ass when it comes to Integers in Pandas. I see only one way out, do use the `apply` method.

In [11]:
objeto.describe()

,Objeto del Gasto
count,1.325511e+06
mean,2.505599e+04
std,1.463928e+04
min,1.100000e+03
25%,1.430200e+04
50%,2.610200e+04
75%,3.510100e+04
max,9.910100e+04


In [12]:
objeto.max()

Objeto del Gasto    99101
dtype: int64

In [13]:
objeto.min()

Objeto del Gasto    1100
dtype: int64

In [14]:
small = objeto[objeto['Objeto del Gasto'] <= 10000]

In [15]:
small.head(n=30)

,Objeto del Gasto
0,6200
1,1100
2,1200
3,1300
4,1400
5,1500
6,1700
7,2100
8,2200
9,2400


In [16]:
len(small)

204326

In [17]:
merged[merged['Objeto del Gasto'] < 10000].groupby(['Ciclo']).count()

,Actividad Institucional,Adefas,Aprobado,Clave de cartera,Descripción de Fuente de Financiamiento,Descripción de Función,Descripción de Grupo Funcional,Descripción de Objeto del Gasto,Descripción de Programa Presupuestario,Descripción de Ramo,...,Modalidad del Programa presupuestario,Modificado,Objeto del Gasto,Pagado,Programa Presupuestario,Ramo,Reasignacion,Subfunción,Tipo de Gasto,Unidad Responsable
Ciclo,,,,,,,,,,,,,,,,,,,,,
2010,129014,0,129014,0,129014,129014,129014,129014,129014,129014,...,129014,0,129014,0,129014,129014,0,129014,129014,129014
2016,75312,0,75312,75312,75312,75312,75312,75312,75312,75312,...,75312,0,75312,0,75312,75312,75312,75312,75312,75312


In [23]:
mx2016 = merged[merged['Ciclo'] == 2016]

In [24]:
mx2016['Objeto del Gasto'].sample(n=10)

28185    4300.0
737      3900.0
10024    6200.0
34794    2900.0
22677    1300.0
67682    1300.0
69606    1200.0
25465    3300.0
34550    1300.0
27396    3300.0
Name: Objeto del Gasto, dtype: float64

In [25]:
mx2010 = merged[merged['Ciclo'] == 2010]

In [31]:
mx2010['Objeto del Gasto'].value_counts()

2101.0    3012
3817.0    2926
3811.0    2533
2106.0    2424
3506.0    2319
3407.0    2214
2301.0    2212
2204.0    2152
1306.0    1860
2602.0    1830
1305.0    1806
1103.0    1783
1509.0    1783
3504.0    1776
3505.0    1758
3106.0    1757
3103.0    1731
3501.0    1712
3808.0    1708
2404.0    1702
3413.0    1652
2701.0    1641
2102.0    1632
1511.0    1621
3101.0    1552
3814.0    1519
3107.0    1487
1413.0    1480
2302.0    1446
2401.0    1431
          ... 
8222.0       1
2600.0       1
8223.0       1
8224.0       1
8225.0       1
8226.0       1
8208.0       1
8207.0       1
8206.0       1
8205.0       1
8120.0       1
8121.0       1
3300.0       1
8122.0       1
8123.0       1
8124.0       1
8125.0       1
8126.0       1
8127.0       1
8128.0       1
8129.0       1
8130.0       1
3200.0       1
8131.0       1
8132.0       1
8201.0       1
8202.0       1
8203.0       1
8204.0       1
8211.0       1
Name: Objeto del Gasto, dtype: int64